Importar Bibliotecas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

Carregar os dados do arquivo .csv

In [ ]:
df = pd.read_csv("MICRODADOS_ENEM_2023.csv", sep=";", encoding="latin1")

Filtrar por colunas relevantes

In [ ]:
df = df[[ 'TP_ESCOLA', 'Q001', 'Q002', 'Q005', 'Q006', 'Q024', 'Q025']]

Renomear colunas relevantes com variáveis

In [ ]:
variaveis = {
    'TP_ESCOLA': 'tipo_escola',
    'Q001': 'formacao_pai',
    'Q002': 'formacao_mae',
    'Q005': 'tamanho_familia',
    'Q006': 'renda_familiar',
    'Q024': 'computador',
    'Q025': 'internet'
}
df = df.rename(columns=variaveis)

Mapear renda familiar e formação academica dos país.


In [ ]:
renda_map = {
    'A': 0,          # Nenhuma renda
    'B': 606,        # Até R$ 1.212,00
    'C': 1515.01,    # R$ 1.212,01 a R$ 1.818,00
    'D': 2121.01,    # R$ 1.818,01 a R$ 2.424,00
    'E': 2727.01,    # R$ 2.424,01 a R$ 3.030,00
    'F': 3333.01,    # R$ 3.030,01 a R$ 3.636,00
    'G': 4242.01,    # R$ 3.636,01 a R$ 4.848,00
    'H': 5454.01,    # R$ 4.848,01 a R$ 6.060,00
    'I': 6666.01,    # R$ 6.060,01 a R$ 7.272,00
    'J': 7878.01,    # R$ 7.272,01 a R$ 8.484,00
    'K': 9090.01,    # R$ 8.484,01 a R$ 9.696,00
    'L': 10302.01,   # R$ 9.696,01 a R$ 10.908,00
    'M': 11514.01,   # R$ 10.908,01 a R$ 12.120,00
    'N': 13332.01,   # R$ 12.120,01 a R$ 14.544,00
    'O': 16362.01,   # R$ 14.544,01 a R$ 18.180,00
    'P': 21210.01,   # R$ 18.180,01 a R$ 24.240,00
    'Q': 30000       # Mais de R$ 24.240,00 (estimado)
}
descricao_formacao = {
    'A': 'Nunca estudou',
    'B': 'Fund. Incompleto',
    'C': 'Fund. Completo',
    'D': 'Médio Incompleto',
    'E': 'Médio Completo',
    'F': 'Superior Completo',
    'G': 'Pós-graduação',
    'H': 'Não sabe'
}

Gráfico de formação acadêmica dos responsáveis

In [ ]:
df['formacao_pai_label'] = df['formacao_pai'].map(descricao_formacao)
df['formacao_mae_label'] = df['formacao_mae'].map(descricao_formacao)
#Concatenando para visualização conjunta
df_responsaveis = pd.concat([
    df['formacao_pai_label'].rename('formacao').to_frame(),
    df['formacao_mae_label'].rename('formacao').to_frame()
])

formacao_counts = df_responsaveis['formacao'].value_counts().reset_index()
formacao_counts.columns = ['formacao', 'frequencia']

ordem_formacao = ['Nunca estudou', 'Fund. Incompleto', 'Fund. Completo',
                  'Médio Incompleto', 'Médio Completo',
                  'Superior Completo', 'Pós-graduação', 'Não sabe']

formacao_counts['formacao'] = pd.Categorical(formacao_counts['formacao'], categories=ordem_formacao, ordered=True)
formacao_counts = formacao_counts.sort_values('formacao')

#Gráfico
fig = px.bar(
    formacao_counts,
    x='formacao',
    y='frequencia',
    title='Formação Acadêmica dos Responsáveis',
    labels={'formacao': 'Formação', 'frequencia': 'Frequência'},
    color_discrete_sequence=['skyblue'],
    template='plotly_white'
)

fig.update_layout(
    xaxis_title='Formação',
    yaxis_title='Frequência'
)

fig.show()

Gráfico de acesso a internet e computador

In [ ]:
map_computador = {
    'A': 'Não tem',
    'B': '1 computador',
    'C': '2 computadores',
    'D': '3 computadores',
    'E': '4 ou mais'
}
map_internet = {
    'A': 'Não tem',
    'B': 'Tem'
}

df['computador_label'] = df['computador'].map(map_computador)
df['internet_label'] = df['internet'].map(map_internet)

acesso_agrupado = df.groupby(['tipo_escola_label', 'internet_label']).size().reset_index(name='frequencia_internet')
computador_agrupado = df.groupby(['tipo_escola_label', 'computador_label']).size().reset_index(name='frequencia_computador')

#Acesso à Internet
fig_internet = px.bar(
    acesso_agrupado,
    x='tipo_escola_label',
    y='frequencia_internet',
    color='internet_label',
    barmode='group',
    title='Acesso à Internet por Tipo de Escola',
    labels={
        'tipo_escola_label': 'Tipo de Escola',
        'frequencia_internet': 'Frequência',
        'internet_label': 'Acesso à Internet'
    },
    template='plotly_white',
    color_discrete_sequence=px.colors.qualitative.Pastel
)

#Acesso a Computador
fig_computador = px.bar(
    computador_agrupado,
    x='tipo_escola_label',
    y='frequencia_computador',
    color='computador_label',
    barmode='group',
    title='Quantidade de Computadores por Tipo de Escola',
    labels={
        'tipo_escola_label': 'Tipo de Escola',
        'frequencia_computador': 'Frequência',
        'computador_label': 'Quantidade de Computadores'
    },
    template='plotly_white',
    color_discrete_sequence=px.colors.qualitative.Pastel1
)

fig_internet.show()
fig_computador.show()

Heatmap de renda familiar X tamanho da familia

In [ ]:
#Limitar gráfico à 10 familias pra remover outliers e manter coesão
df_familia = df[(df['renda_familiar'].notna()) & (df['tamanho_familia'] <= 10)]

heatmap_data = df_familia.groupby(
    ['tamanho_familia', 'renda_familiar']
).size().reset_index(name='frequencia')

heatmap_data['tamanho_familia'] = heatmap_data['tamanho_familia'].astype(str)

#Gráfico
fig = px.density_heatmap(
    heatmap_data,
    x='tamanho_familia',
    y='renda_familiar',
    z='frequencia',
    histfunc='sum',
    title='Distribuição: Tamanho da Família vs. Renda Familiar (até 10 pessoas)',
    labels={
        'tamanho_familia': 'Número de Pessoas na Residência',
        'renda_familiar': 'Renda Familiar (R$)',
        'frequencia': 'Frequência da Renda'
    },
    color_continuous_scale='YlGnBu',
    template='plotly_white'
)
fig.update_coloraxes(colorbar_title='')
fig.update_layout(
    yaxis_title='Renda Familiar por Grupos (estimada)',
    xaxis_title='Tamanho da Família'
)

fig.show()

Gráfico formzação dos pais X tipo escolar do aluno

In [ ]:

df_formacao = pd.concat([
    df[['formacao_pai_label', 'tipo_escola_label']].rename(columns={'formacao_pai_label': 'formacao'}),
    df[['formacao_mae_label', 'tipo_escola_label']].rename(columns={'formacao_mae_label': 'formacao'})
])
formacao_escola = df_formacao.groupby(['tipo_escola_label', 'formacao']).size().reset_index(name='frequencia')

ordem_formacao = [
    'Nunca estudou', 'Fund. Incompleto', 'Fund. Completo',
    'Médio Incompleto', 'Médio Completo',
    'Superior Completo', 'Pós-graduação', 'Não sabe'
]
formacao_escola['formacao'] = pd.Categorical(
    formacao_escola['formacao'],
    categories=ordem_formacao,
    ordered=True
)
formacao_escola = formacao_escola.sort_values('formacao')

#Gráfico
fig = px.bar(
    formacao_escola,
    x='formacao',
    y='frequencia',
    color='tipo_escola_label',
    barmode='group',
    title='Formação dos Responsáveis por Tipo de Escola',
    labels={
        'formacao': 'Nível de Escolaridade',
        'frequencia': 'Frequência',
        'tipo_escola_label': 'Tipo de Escola'
    },
    template='plotly_white',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_layout(
    xaxis_title='Formação dos Responsáveis',
    yaxis_title='Frequência',
    legend_title='Tipo de Escola'
)

fig.show()